In [1]:
# from llama import Llama
# from typing import List
import yaml
from pyprojroot import here
# from pydantic import create_model
# import inspect, json
# from inspect import Parameter
from transformers import LlamaForCausalLM, LlamaTokenizer
from transformers import (
    # AutoModelForCausalLM,
    # AutoTokenizer,
    BitsAndBytesConfig,
    # pipeline
    )
import sentencepiece
import torch
from typing import List
import torch
from tqdm import tqdm

In [2]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)

def prepare_prompt_for_llama(prompt, tokenizer):
    prompt = f"<s>[INST]{prompt}[/INST]"
    return tokenizer(prompt, return_tensors="pt")

CUDA available: True
CUDA version: 12.1


In [3]:
with open(here("configs/config.yml")) as cfg:
        app_config = yaml.load(cfg, Loader=yaml.FullLoader)
llama2_7b_chat_dir = here(app_config["model_dir"]["llama2_7b_chat"])
tokenizer_dir = here(app_config["model_dir"]["llama2_7b_chat"])
max_seq_len = app_config["llama_cfg"]["max_seq_len"]
max_batch_size = app_config["llama_cfg"]["max_batch_size"]

tokenizer = LlamaTokenizer.from_pretrained(tokenizer_dir, padding_side="right")
model = LlamaForCausalLM.from_pretrained(
    llama2_7b_chat_dir,
    device_map="cuda",
    )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

c:\Users\farza\anaconda3\envs\AR-RT-LLM\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
prompt = "Who is Elon Musk?"
result = model.generate(**prepare_prompt_for_llama(prompt, tokenizer).to("cuda"), max_new_tokens=150)
print(tokenizer.batch_decode(result))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s><s>[INST]Who is Elon Musk?[/INST]  Elon Musk is a South African-born entrepreneur, inventor, and business magnate who is best known for his ambitious goals in revolutionizing transportation, energy, and space exploration. He is the CEO of SpaceX and Tesla, Inc., and has been involved in other successful ventures such as Neuralink and The Boring Company.\nMusk was born in 1971 in Pretoria, South Africa. He developed an interest in computing and programming at an early age and taught himself computer programming. He moved to Canada in 1992 to attend college, and later transferred to the University of Pennsylvania, where he graduated with a degree in economics and']


In [ ]:
prompt = "What is the capital of France?"
result = model.generate(**prepare_prompt_for_llama(prompt, tokenizer).to("cuda"), max_new_tokens=150)
print(tokenizer.batch_decode(result, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["[INST]What is the capital of France?[/INST]  The capital of France is Paris. Located in the Île-de-France region, Paris is the country's largest city and is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also the political, cultural, and economic center of France, home to many international organizations and embassies."]


In [8]:
prompt = "Tell me how to train my dog to sit"
result = model.generate(**prepare_prompt_for_llama(prompt, tokenizer).to("cuda"), max_new_tokens=150)
print(tokenizer.batch_decode(result))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


["<s><s>[INST]Tell me how to train my dog to sit[/INST]  Training your dog to sit is a basic obedience command that can be achieved with patience, consistency, and positive reinforcement. Here's a step-by-step guide on how to train your dog to sit:\n1. Choose a quiet and distraction-free area: Find a quiet area with no distractions where your dog can focus on you.\n2. Have treats ready: Choose your dog's favorite treats and have them ready to use as rewards.\n3. Stand in front of your dog: Stand in front of your dog and hold a treat close to their nose.\n4. Move the treat up and back: Slowly move the treat up and"]
